In [1]:
import pandas as pd
train = pd.read_csv("si630w22-hw3-train.csv")
dev = pd.read_csv("si630w22-hw3-dev.csv")
qa = pd.read_csv("si630w22-hw3-data.csv")
qa["raw_data"] = qa["question_text"] + qa["reply_text"]
def find_qa_text(question_id):
    try:
        return qa[qa['question_id'] == question_id].iloc[0].raw_data
    except:
        pass
train["qa_text"] = train["id"].apply(find_qa_text)
train["text"] = train["qa_text"] + train["annotator_id"] + train["group"]
dev["qa_text"] = dev["id"].map(find_qa_text)
dev["text"] = dev["annotator_id"] + dev["group"] + dev["qa_text"] 

In [2]:
train_labels = train["rating"]
test_labels = dev["rating"]

In [3]:
train_texts, train_labels = train["text"].tolist(), train["rating"].tolist()
test_texts, test_labels = dev["text"].tolist(), dev["rating"].tolist()

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/MiniLM-L12-H384-uncased")
max_length = 512
train_encodings = tokenizer(train_texts, truncation=True, padding="max_length", max_length=max_length)
test_encodings = tokenizer(test_texts, truncation=True, padding="max_length", max_length=max_length)
small_train_encodings = tokenizer(train_texts[0:100], truncation=True, padding="max_length", max_length=max_length)
small_test_encodings = tokenizer(test_texts[0:100], truncation=True, padding="max_length", max_length=max_length)

In [5]:
import torch

class HelpfulnessDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = HelpfulnessDataset(train_encodings, train_labels)
test_dataset = HelpfulnessDataset(test_encodings, test_labels)

In [6]:
small_train_dataset = HelpfulnessDataset(small_train_encodings, train_labels[0:100])
small_test_dataset = HelpfulnessDataset(small_test_encodings, test_labels[0:100])

In [7]:
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # calculate accuracy using sklearn's function
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }

In [9]:
from transformers import Trainer, TrainingArguments
from transformers import AutoModelForSequenceClassification
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=40,
)

model = AutoModelForSequenceClassification.from_pretrained("microsoft/Multilingual-MiniLM-L12-H384")

trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=small_train_dataset,         # training dataset
    eval_dataset=small_test_dataset             # evaluation dataset
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/Multilingual-MiniLM-L12-H384 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\fangz\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 100
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 7


RuntimeError: CUDA out of memory. Tried to allocate 192.00 MiB (GPU 0; 4.00 GiB total capacity; 2.50 GiB already allocated; 0 bytes free; 2.53 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [10]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 100
  Batch size = 64


RuntimeError: CUDA out of memory. Tried to allocate 48.00 MiB (GPU 0; 4.00 GiB total capacity; 2.55 GiB already allocated; 0 bytes free; 2.57 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF